In [ ]:
import pandas as pd
import sqlite3

In [ ]:
# Load the dataset
df = pd.read_csv("data/RealEstate_California.csv")

# Drop rows where hasBadGeocode is 1 (true)
df = df[df["hasBadGeocode"] != 1]

# Drop unneeded columns
df = df.drop(columns=['stateId', 'cityId', 'countyId', 'is_bankOwned', 'description', 'hasBadGeocode', 'is_forAuction', 'country', 
            'currency', 'state', 'id', 'time', 'levels', 'livingAreaValue', 
            'lotAreaUnits', 'buildingArea', 'hasGarage'])


In [ ]:
# Convert DataFrame columns to appropriate types
df["id"] = pd.to_numeric(df["Unnamed: 0"], errors="coerce").fillna(0).astype(int)
df["datePostedString"] = df["datePostedString"].astype(str)
df["price"] = pd.to_numeric(df["price"], errors="coerce").fillna(0).astype(int)
df["pricePerSquareFoot"] = pd.to_numeric(df["pricePerSquareFoot"], errors="coerce").fillna(0).astype(int)
df["bedrooms"] = pd.to_numeric(df["bedrooms"], errors="coerce").fillna(0).astype(int)
df["bathrooms"] = pd.to_numeric(df["bathrooms"], errors="coerce").fillna(0)
df["streetAddress"] = df["streetAddress"].astype(str)
df["city"] = df["city"].astype(str)
df["zipcode"] = df["zipcode"].astype(str)  # ZIP codes should be text
df["latitude"] = pd.to_numeric(df["latitude"], errors="coerce")
df["longitude"] = pd.to_numeric(df["longitude"], errors="coerce")
df["yearBuilt"] = pd.to_numeric(df["yearBuilt"], errors="coerce").fillna(0).astype(int)
df["livingArea"] = pd.to_numeric(df["livingArea"], errors="coerce").fillna(0).astype(int)
df["parking"] = df["parking"].astype(bool)
df["garageSpaces"] = pd.to_numeric(df["garageSpaces"], errors="coerce").fillna(0).astype(int)
df["pool"] = df["pool"].astype(bool)
df["spa"] = df["spa"].astype(bool)
df["isNewConstruction"] = df["isNewConstruction"].astype(bool)
df["hasPetsAllowed"] = df["hasPetsAllowed"].astype(bool)
df["homeType"] = df["homeType"].astype(str)
df["county"] = df["county"].astype(str)
df["event"] = df["event"].astype(str)


In [ ]:
df = df.reset_index(drop=True)

data = [
    (
        row["Unnamed: 0"], row["datePostedString"], row["price"], row["pricePerSquareFoot"],
        row["bedrooms"], row["bathrooms"], row["streetAddress"],
        row["city"], row["zipcode"], row["latitude"], row["longitude"],
        row["yearBuilt"], row["livingArea"], row["parking"],
        row["garageSpaces"], row["pool"], row["spa"],
        row["isNewConstruction"], row["hasPetsAllowed"],
        row["homeType"], row["county"], row["event"]
    )
    for _, row in df.iterrows()
]

In [ ]:
# Connect to SQLite and create real_estate database
conn = sqlite3.connect("real_estate.sqlite")

# Drop real_estate table if it exists
cursor = conn.cursor()
cursor.execute("DROP TABLE IF EXISTS real_estate")


In [ ]:
# Create the real_estate table
cursor.execute("""
CREATE TABLE IF NOT EXISTS real_estate (
    id INTEGER PRIMARY KEY,
    datePostedString TEXT,
    price INTEGER,
    pricePerSquareFoot INTEGER,
    bedrooms INTEGER,
    bathrooms REAL,
    streetAddress TEXT,
    city TEXT,
    zipcode TEXT,
    latitude REAL,
    longitude REAL,
    yearBuilt INTEGER,
    livingArea INTEGER,
    parking BOOLEAN,
    garageSpaces INTEGER,
    pool BOOLEAN,
    spa BOOLEAN,
    isNewConstruction BOOLEAN,
    hasPetsAllowed BOOLEAN,
    homeType TEXT,
    county TEXT,
    event TEXT
);
""")

In [ ]:
# Query to bulk insert data into the real_estate table
sql = """
INSERT INTO real_estate (
    id, datePostedString, price, pricePerSquareFoot, bedrooms, bathrooms, streetAddress, 
    city, zipcode, latitude, longitude, yearBuilt, livingArea, 
    parking, garageSpaces, pool, spa, isNewConstruction, hasPetsAllowed, homeType, county, event
) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?);
"""

In [ ]:
# Insert the data into the real_estate table
cursor.executemany(sql, data)
conn.commit()

In [ ]:
# Test query to grab first 5 from the database
query = "SELECT * FROM real_estate LIMIT 5"
df_test = pd.read_sql_query(query, conn)

conn.close()

df_test.head()

In [ ]:
# TODO: more queries to test the data and glean insights